In [12]:
# Pyton Package Installation

## virtual environment setup
%pip install conda

## nlp libraries
%pip install --upgrade gensim
%pip install top2vec

## machine learning libraries
%pip install transformers
%pip install torch

## visualization libraries
%pip install matplotlib
%pip install seaborn
%pip install pyldavis
pip install cartopy

## datahandling libraries
%pip install pandas

SyntaxError: invalid syntax (1209796400.py, line 18)

In [ ]:
# NLP-Pipeline
import nltk 
import sklearn
import gensim
# import spaCy as sp
import re

# ML
import transformers
import torch 

## Embeddings

import top2vec as t2v 

## Visualisierung
import matplotlib as mpl
%matplotlib inline
import matplotlib.pyplot as plt
import seaborn as sns
import wordcloud as wc
import pyLDAvis
import cartopy as ctp

# Sortieren
import venv
import numpy as np

TF-IDF

In [ ]:
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.decomposition import LatentDirichletAllocation

# Import des Datensatzes
complaints = [complaints_data_cleanced.csv]
    "I am very unhappy with the service I received.",
    "The product quality is excellent and exceeded my expectations.",
    "Customer support was unhelpful and rude.",
    "Delivery was prompt and the packaging was secure.",
    "I had a terrible experience with the billing department."
]
#TF-IDF + LDA

vectorizer = TfidfVectorizer(max_df=0.6, min_df=1, stop_words='english')
tfidf_matrix = vectorizer.fit_transform(complaints)
lda = LatentDirichletAllocation(n_components=10)
lda_topics = lda.fit_transform(tfidf_matrix)

print(lda_topics)

[[0.6705771  0.03660255 0.03660254 0.03660255 0.03660255 0.03660254
  0.03660255 0.03660254 0.03660255 0.03660255]
 [0.72188466 0.03090171 0.0309017  0.03090171 0.03090171 0.0309017
  0.03090171 0.0309017  0.03090171 0.03090171]
 [0.03333333 0.03333334 0.69999997 0.03333334 0.03333334 0.03333334
  0.03333334 0.03333334 0.03333334 0.03333334]
 [0.03333333 0.03333334 0.03333334 0.03333334 0.03333334 0.03333334
  0.03333334 0.69999997 0.03333334 0.03333334]
 [0.03333333 0.03333334 0.03333334 0.03333334 0.03333334 0.69999997
  0.03333334 0.03333334 0.03333334 0.03333334]]


In [4]:
%pip install spacy
%pip install https://github.com/explosion/spacy-models/releases/download/en_core_web_sm-3.7.1/en_core_web_sm-3.7.1-py3-none-any.whl
%pip uninstall transformers torch sentencepiece -y

Note: you may need to restart the kernel to use updated packages.
  Using cached https://github.com/explosion/spacy-models/releases/download/en_core_web_sm-3.7.1/en_core_web_sm-3.7.1-py3-none-any.whl (12.8 MB)
Note: you may need to restart the kernel to use updated packages.
Found existing installation: transformers 4.57.5
Uninstalling transformers-4.57.5:
  Successfully uninstalled transformers-4.57.5
Found existing installation: torch 2.9.1
Uninstalling torch-2.9.1:
  Successfully uninstalled torch-2.9.1
Found existing installation: sentencepiece 0.2.1
Uninstalling sentencepiece-0.2.1:
  Successfully uninstalled sentencepiece-0.2.1
Note: you may need to restart the kernel to use updated packages.


# Datenvorverarbeiten (engl. data pre-processing)

In [20]:
# Landen des bereinigten Datensatzes
df = pd.read_csv('/Users/kaiweber/Documents/GitHub/DLBDSEDA02_Projekt_Advanced-Data-Analysis/src/complaints_data_cleaned.csv', usecols=["text"])
print(df.head(10))
print(df.shape)

                                                text
0  I used to love Comcast. Until all these consta...
1  I'm so over Comcast! The worst internet provid...
2  If I could give them a negative star or no sta...
3  I've had the worst experiences so far since in...
4  Check your contract when you sign up for Comca...
5  Thank God. I am changing to Dish. They gave me...
6  I Have been a long time customer and only have...
7  There is a malfunction on the DVR manager whic...
8  Charges overwhelming. Comcast service rep was ...
9  I have had cable, DISH, and U-verse, etc. in t...
(5627, 1)


In [32]:
import pandas as pd
import spacy

# Textbereinigung (engl. text cleaning)
##  Rauschentfernung (engl. noise reduction)
### Wortbereinigung (engl. word cleaning)

# Datensatz laden (Rohtext)
df = pd.read_csv('/Users/kaiweber/Documents/GitHub/DLBDSEDA02_Projekt_Advanced-Data-Analysis/src/complaints_data_cleaned.csv', 
                 usecols=["text"],nrows=150)                                # Zeilenbegrenzung

# Linguistische Verarbeitung (engl. linguistic processing)
## Rechtschreibfehlerkorrektur (engl. spelling correction)

## Textbereinigung, Tokenisierung, Lemmatisierung

##mit spaCy Pipeline (englisches Modell (small version))
nlp = spacy.load("en_core_web_sm")

## Textbereinigung, Tokenisierung, Lemmatisierung
df["cleaned"] = [[token.lemma_.lower()
                  for token in doc
                  # Filterblock      
                  if (not token.is_stop and                                 # Standard:     Stopwort-Filter
                      not token.is_punct and                                # Standard:     Satzzeichen-Filter
                      not token.like_num and                                # Standard:     Nummern-Filter (einfache Zahlen)
                      len(token.text) > 2 and                               # Individuell:  Wörter mit min. 2 Zeichen
                      not any(char in token.text for char in ':/-–—') and   # Individuell:  Filter für Datums-/Zeit-Token
                      token.is_ascii and                                    # Individuell:  Emoijs-Filter (True = AN / False = Nur Emoijs)
                      token.pos_ != "PRON" and                              # Individuell:  Pronomen-Filter
                      token.text.lower() not in ["meh", "ugh"])]            # Individuell:  Wortfilter
                 for doc in nlp.pipe(df["text"], batch_size=50)]            # Batch-Verarbeitung (50 Texte parallel)

print(f"Verarbeitet: {len(df)} Beschwerden")

# Ausgabe des Prozesses
df.head(40)[["text", "cleaned"]].style.set_properties(
    **{'text-align': 'left', 'width': '1000px', 'max-width': '1500px', 'font-size': '12px'}
)

Verarbeitet: 150 Beschwerden


,text,cleaned
0,"I used to love Comcast. Until all these constant updates. My internet and cable crash a lot at night, and sometimes during the day, some channels don't even work and on demand sometimes don't play either. I wish they will do something about it. Because just a few mins ago, the internet have crashed for about 20 mins for no reason. I'm tired of it and thinking about switching to Wow or something. Please do not get Xfinity.","['love', 'comcast', 'constant', 'update', 'internet', 'cable', 'crash', 'lot', 'night', 'day', 'channel', 'work', 'demand', 'play', 'wish', 'min', 'ago', 'internet', 'crash', 'min', 'reason', 'tired', 'think', 'switch', 'wow', 'xfinity']"
1,I'm so over Comcast! The worst internet provider. I'm taking online classes and multiple times was late with my assignments because of the power interruptions in my area that lead to poor quality internet service. Definitely switching to Verizon. I'd rather pay $10 extra then dealing w/ Comcast and non stopping internet problems.,"['comcast', 'bad', 'internet', 'provider', 'take', 'online', 'class', 'multiple', 'time', 'late', 'assignment', 'power', 'interruption', 'area', 'lead', 'poor', 'quality', 'internet', 'service', 'definitely', 'switch', 'verizon', 'pay', 'extra', 'deal', 'comcast', 'non', 'stop', 'internet', 'problem']"
2,"If I could give them a negative star or no stars on this review I would. I have never worked with any industry with as bad of customer service as Comcast. It is not a matter of money because I make well enough above and beyond to afford their services but they are a legitimate ripoff. I think they are the biggest scam of since the mortgage industry's major meltdown and I hope I move somewhere where Comcast does not exist. The disregard to want to help or do the right thing is honestly astounding. If you have to call, which you do FOR ALL ISSUES - billing, connection/service, adding or removing service, errors, it does not matter you will be transferred minimum of 4 times. Everyone says the same thing and passes the issues to the next person and no one resolves the problem.They offer promotional packages in small timeframes and can never access them again so they then upgrade you without you wishing and change your billing. It has been 5 months and I have been overcharged $40 a month since I started with them. The blatant rudeness that must make you qualified to do this job is the type of quality service that gets you this review. So... Dear Comcast, you suck. Sincerely, a customer who cannot wait to never use your service again.","['negative', 'star', 'star', 'review', 'work', 'industry', 'bad', 'customer', 'service', 'comcast', 'matter', 'money', 'afford', 'service', 'legitimate', 'ripoff', 'think', 'big', 'scam', 'mortgage', 'industry', 'major', 'meltdown', 'hope', 'comcast', 'exist', 'disregard', 'want', 'help', 'right', 'thing', 'honestly', 'astounding', 'issues', 'billing', 'connection', 'service', 'add', 'remove', 'service', 'error', 'matter', 'transfer', 'minimum', 'time', 'say', 'thing', 'pass', 'issue', 'person', 'resolve', 'problem', 'offer', 'promotional', 'package', 'small', 'timeframe', 'access', 'upgrade', 'wish', 'change', 'billing', 'month', 'overcharge', 'month', 'start', 'blatant', 'rudeness', 'qualified', 'job', 'type', 'quality', 'service', 'get', 'review', 'dear', 'comcast', 'suck', 'sincerely', 'customer', 'wait', 'use', 'service']"
3,"I've had the worst experiences so far since install on 10/4/16. Nothing but problems. Two no shows on scheduled service appointments, extreme difficulty in adding boxes to the second floor. What is so difficult about adding boxes to an existing account? No thank you, I'm not starting a second account for the second floor of the same house! A separate bundle package? All I wanted was just to add a few boxes. Apparently this is not possible. Well then, I guess it's not possible to remain a customer!","['bad', 'experience', 'far', 'install', 'problem', 'show', 'schedule', 'ser

# Vokabularerstellung (engl. vocabulary construction)
Mapping der Wörter (Token) zu IDs aus den bereinigten Beschwerden.

Wortfrequenzschwellen
ggf. nur Nomen?

In [ ]:
# Vokabularerstellung (engl. vocabulary construction)
from sklearn.feature_extraction.text import CountVectorizer

# cleaned-Listen zu Strings joinen (Vectorizer braucht Strings)
df["text_cleaned"] = [" ".join(tokens) for tokens in df["cleaned"]]

vectorizer = CountVectorizer()                                                  # Vectorizer mit fit_transform: Vokabular + Matrix in einem
X = vectorizer.fit_transform(df["text_cleaned"])

## Vokabularerstellung
vocabulary = vectorizer.get_feature_names_out()                                 # Vokabular extrahieren
print(f"Vokabular-Größe: {len(vocabulary)} Token (Wörter)")                     # Ausgabe der Vokabulargrö0e

word_counts = X.sum(axis=0).A1                                                  # Häufigkeiten (Summe pro Spalte)
vocab_df = pd.DataFrame({
    'word': vocabulary,
    'frequency': word_counts
}).sort_values('frequency', ascending=False)

vocab_df.head(50)                                                               # Erste Spalte ID


Vokabular-Größe: 1870 Token (Wörter)


,word,frequency
1476,service,295
295,comcast,276
864,internet,203
1646,tell,148
1674,time,138
399,customer,138
228,call,129
1046,month,105
1446,say,103
1166,pay,97


# Textbereinigung (engl. text cleaning)

##  Rauschentfernung (engl. noise reduction)
### Wortbereinigung (engl. word cleaning)
#### Stoppworte (engl. stopwords)
- individuelle Stopworte (Beschwerde)
### Zeichenbereinigung (engl. character cleaning)
#### Satzzeichen (engl. punctuation marks)
#### Leerzeichen (engl. white space)
#### Sonderzeichen (engl. special character)
### Nummernbereinigung (engl. numbers cleaning)
#### Nummern (engl. removing numbers)


##  Standardisierung (engl. standardisation)
### Normalisierung (engl. normalisation)
#### Kasusumwandlung (engl. case conversion)
#### Formatnormalisierungen (engl. format normalisations)
### Rechtschreibfehlerkorrektur (engl. spelling correction)
auf eine Rechtschreikorrektur wird verzichtet, da contextualSpellcheck sicht nicht installieren ließ (Mac M1), T5 zu lange dauerte 10min + und PyEnchant schlechte Korrekturen lieferte

# lexikalische Verarbeitung (engl. lexical processing)
##  Tokenisierung (engl. tokenization)
##  Grundformreduktion (engl. inflection reduction)
### Lemmatisierung (engl. lemmatization)
NLTK(WordNetLemmatizer) / SpaCy
##  Vokabularerstellung/Wortschatzaufbau (engl. vocabulary construction)
Scikit-learn (CountVectorizer)
fit_transform lernt das Vokabular / dict und gibt eine Matrix zurück
# syntaktische Verarbeitung (engl. syntactic processing)
# semantische Verarbeitung (engl. context processing)
## Semantisches Parsen (engl. semantic parsing)
### Eigennamenerkennung (engl. Named Entity Recognition - NER)

# syntaktische Verarbeitung (engl. syntactic processing)
wird verzichtet.

# Sortieren / Lemmatisierung 

In [22]:
import pandas as pd
import spacy

# 1. CSV laden

# 2. spaCy einmalig laden
nlp = spacy.load("en_core_web_sm")

# 3. Batch-Lemmatisierung (schnell!)
df["lemmas"] = [[token.lemma_.lower() 
                 for token in doc 
                 if not token.is_stop and not token.is_punct 
                 and not token.like_num]
                for doc in nlp.pipe(df["text"], batch_size=50)]

print(df.head())
print(f"Verarbeitet: {len(df)} Beschwerden")

                                                text  \
0  I used to love Comcast. Until all these consta...   
1  I'm so over Comcast! The worst internet provid...   
2  If I could give them a negative star or no sta...   
3  I've had the worst experiences so far since in...   
4  Check your contract when you sign up for Comca...   

                                             cleaned  \
0  [love, comcast, constant, update, internet, ca...   
1  [comcast, bad, internet, provider, take, onlin...   
2  [negative, star, star, review, work, industry,...   
3  [bad, experience, far, install, problem, show,...   
4  [check, contract, sign, comcast, advertised, o...   

                                              lemmas  
0  [love, comcast, constant, update, internet, ca...  
1  [comcast, bad, internet, provider, take, onlin...  
2  [negative, star, star, review, work, industry,...  
3  [bad, experience, far, install, 10/4/16, probl...  
4  [check, contract, sign, comcast, advertised, o..